In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import fft
import wave
import sys
import os

from scipy.io.wavfile import write
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.io import wavfile
import scipy.signal as signal
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


In [ ]:
path = 'C:/Users/User/Desktop/AppStat/MachineLearning/AppliedML2024/final_project/data/nsynth-valid/audio/'

In [ ]:
def read_files_in_dir(directory):
    filenames = os.listdir(directory)
    return filenames


def audio_to_waveform(audio):
    waveform, sr = librosa.load(audio, sr=None)
    return waveform, sr

def waveform_to_spectogram(waveform):
    if waveform.dtype != np.float32:
        waveform = waveform.astype(np.float32)
    spectrogram = librosa.stft(waveform)
    return np.abs(spectrogram)

def pad_spectrogram(spec, target_shape):
    padded_spec = np.zeros(target_shape,dtype = np.float32)
    min_shape = np.minimum(target_shape, spec.shape)
    padded_spec[:min_shape[0], :min_shape[1]] = spec[:min_shape[0], :min_shape[1]]
    return padded_spec

def spectogram_to_audio(spectrogram, sr,output_wav):
    waveform = librosa.istft(spectrogram)
    waveform = np.nan_to_num(waveform)
    waveform = waveform/np.max(np.abs(waveform))
    return write(output_wav, sr, (waveform*32767).astype(np.int16))

def normalize_spectrogram(spectrogram):
    min_val = np.min(spectrogram)
    max_val = np.max(spectrogram)
    normalized_spectrogram = (spectrogram - min_val) / (max_val - min_val + 1e-6)
    return normalized_spectrogram

def pick_samples_and_classify(arrays):
    #Picks a random number of samples, and returns their filepath and label
    instruments = []
    #pick at minimum two instruments
    number_of_instruments = 2
    np.random.randint(2, len(arrays) + 1)
    labels = np.zeros(len(arrays))
    already_picked = []

    while len(instruments) < number_of_instruments:
        random_pick = np.random.randint(0, len(arrays))
        if random_pick in already_picked:
            break
        else: 
            already_picked.append(random_pick)
            pick = np.random.choice(arrays[random_pick], 1)
            instruments.append(pick)
            labels[random_pick] = 1

    return instruments, labels

#read the filenames, and add their data to 5 lists
def add_waveform_to_list(filenames):
    waveforms = []
    for filename in filenames:
        waveform, params = audio_to_waveform(path + filename[0])
        waveforms.append(waveform)
    return waveforms

def find_longest_array(arrays):
    longest = 0
    for array in arrays:
        if len(array) > longest:
            longest = len(array)
    return longest

def combine_waveforms(waveforms):
    normalization = 1 / len(waveforms)
    # changed to be equal to the length of the longest waveform
    out = np.zeros(find_longest_array(waveforms), dtype=np.float32)
    for w in waveforms:
        out += w.astype(np.float32) * normalization
    return out

# Function to plot spectrogram using librosa
def plot_spectrogram(waveform, sr, title):
    # Compute the STFT
    D = librosa.stft(waveform)
    # Convert to magnitude
    D_mag = np.abs(D)
    print(D_mag.shape)
    # Plot the spectrogram
    plt.figure(figsize=(10, 6))
    librosa.display.specshow(librosa.amplitude_to_db(D_mag, ref=np.max), sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

def plot_separated_spectrograms(output_tensor, sr):
    for i, spec in enumerate(output_tensor):
        plt.figure(figsize=(15, 5))
        librosa.display.specshow(librosa.amplitude_to_db(spec, ref=np.max), sr=sr, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        plt.title(f'Separated Spectrogram {i + 1}')
        plt.show()

def torch_normalize(spectogram):
    min_val = torch.min(spectogram)
    max_val = torch.max(spectogram)
    normalized_spectogram = (spectogram - min_val) / (max_val - min_val)
    return normalized_spectogram

In [ ]:
def nu_gen_spectro(N, target_shape=(1025, 126)):
    data = []
    labels = []
    original_labels = []
    #instrument_list = [bass, guitar, flutes, keyboards]
    instrument_list = [bass_electronic, flute_acoustic]

    for i in range(N):
        paths, label = pick_samples_and_classify(instrument_list)
        original_labels.append(label)
        waveforms = add_waveform_to_list(paths)
        mixed_waveform = combine_waveforms(waveforms)
        
        #audio = audio_to_waveform(mixed_waveform)
        mixed_spectro = waveform_to_spectogram(mixed_waveform)
        #mixed_spectro = audio_to_spectrogram(mixed_waveform)
        mixed_spectro_padded = pad_spectrogram(mixed_spectro, target_shape)
        mixed_spectro_normalized = normalize_spectrogram(mixed_spectro_padded)
        
        inter_waveforms = []
        inst_i = 0
        for n, i in enumerate(label):
            if i == 1:
                spectro = waveform_to_spectogram(waveforms[inst_i])
                spectro_padded = pad_spectrogram(spectro, target_shape)
                spectro_normalized = normalize_spectrogram(spectro_padded)
                inter_waveforms.append(spectro_normalized)
                inst_i += 1

            if i == 0:
                inter_waveforms.append(np.zeros(target_shape))

        data.append(mixed_spectro_normalized)
        #inter_waveforms.append(np.zeros(target_shape))
        labels.append(inter_waveforms)
    
    data = np.array(data)
    
    return data, np.array(labels), np.array(original_labels)

In [ ]:
filenames = read_files_in_dir(path)
#pianos = [filename for filename in filenames if "piano" in filename] #empty
bass = [filename for filename in filenames if "bass" in filename]
guitar = [filename for filename in filenames if "guitar" in filename]
#drum = [filename for filename in filenames if "drum" in filename] #empty
flutes = [filename for filename in filenames if "flute" in filename]
keyboards = [filename for filename in filenames if "keyboard" in filename] 
guitar_acoustic = [filename for filename in filenames if "guitar_acoustic" in filename]
flute_acoustic = [filename for filename in filenames if "flute_acoustic" in filename]
bass_electronic = [filename for filename in filenames if "bass_electronic" in filename]

In [ ]:
target_shape = (1025, 126)
N = 3000  # Number of samples


data, labels, original_labels = nu_gen_spectro(N, target_shape)

print(np.shape(data))  # Should be (N, 1025, 128)
print(np.shape(labels))

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
class UNet(nn.Module):
    def __init__(self, input_channels=1, output_channels=2):
        super(UNet, self).__init__()

        # Define the contracting/downsampling path
        self.conv1 = nn.Conv2d(input_channels, 8, (3,125), padding=(1,62)) # Maybe different kernel and padding.
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.dropout = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)

        # Add transpose convolutional layers for upsampling
        self.upconv1 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv8 = nn.Conv2d(512, 256, 3, padding=1)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv9 = nn.Conv2d(256, 128, 3, padding=1)
        self.upconv3 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv10 = nn.Conv2d(128, 64, 3, padding=1)
        self.upconv4 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.conv11 = nn.Conv2d(64, 32, 3, padding=1)
        self.upconv5 = nn.ConvTranspose2d(32, 16, 2, stride=2)
        self.conv12 = nn.Conv2d(32, 16, 3, padding=1)
        self.upconv6 = nn.ConvTranspose2d(16, 8, 2, stride=2)
        self.conv13 = nn.Conv2d(16, 8, 3, padding=1)
        self.final_conv = nn.Conv2d(8, output_channels, 3,padding = 1)
    def forward(self, x):
        # Contracting path
        x = self.dropout(x)
        x1 = nn.functional.relu(self.bn1(self.conv1(x)))
        x2 = self.pool(nn.functional.relu(self.bn2(self.conv2(x1))))
        x3 = self.pool(nn.functional.relu(self.bn3(self.conv3(x2))))
        x4 = self.pool(nn.functional.relu(self.bn4(self.conv4(x3))))
        x5 = self.pool(nn.functional.relu(self.bn5(self.conv5(x4))))
        x6 = self.pool(nn.functional.relu(self.bn6(self.conv6(x5))))
        x7 = self.pool(nn.functional.relu(self.bn7(self.conv7(x6))))
        # Print sizes for debugging
        #print(f"x1 shape: {x1.shape}")
        #print(f"x2 shape: {x2.shape}")
        #print(f"x3 shape: {x3.shape}")
        #print(f"x4 shape: {x4.shape}")
        #print(f"x5 shape: {x5.shape}")
        #print(f"x6 shape: {x6.shape}")
        #print(f"x7 shape: {x7.shape}")
        x = self.upconv1(x7)
        x = F.interpolate(x, size=(x6.size(2), x6.size(3)), mode='nearest')
    
        #print(f"x shape before concatenation: {x.shape}")
        x = torch.cat([x, x6], dim=1)
        #print(f"x shape after concatenation: {x.shape}")
        x = nn.functional.relu(self.conv8(x))

        x = self.upconv2(x)
        x = F.interpolate(x, size=(x5.size(2), x5.size(3)), mode='nearest')
                #print(f"x shape before concatenation: {x.shape}")
        x = torch.cat([x, x5], dim=1)
        #print(f"x shape after concatenation: {x.shape}")
        x = nn.functional.relu(self.conv9(x))

        x = self.upconv3(x)
        x = F.interpolate(x, size=(x4.size(2), x4.size(3)), mode='nearest')
        #print(f"x shape before concatenation: {x.shape}")
        x = torch.cat([x, x4], dim=1)
        #print(f"x shape before concatenation: {x.shape}")
        x = nn.functional.relu(self.conv10(x))
        
        x = self.upconv4(x)
        x = F.interpolate(x, size=(x3.size(2), x3.size(3)), mode='nearest')
        #print(f"x2_interp shape: {x3.shape}")
        #print(f"x shape before concatenation: {x.shape}")
    
        x = torch.cat([x, x3], dim=1)
        #print(f"x shape before concatenation: {x.shape}")
        x = nn.functional.relu(self.conv11(x))

        x = self.upconv5(x)
        x = F.interpolate(x, size=(x2.size(2), x2.size(3)), mode='nearest')
        #print(f"x shape before concatenation: {x.shape}")
        x = torch.cat([x, x2], dim=1)
        #print(f"x shape before concatenation: {x.shape}")
        x = nn.functional.relu(self.conv12(x))
        x = self.upconv6(x)
        #print(f'upconv6 shape: {x.shape}')
        x = F.interpolate(x, size=(x1.size(2), x1.size(3)), mode='nearest')
        x = torch.cat([x, x1], dim=1)
        x = F.relu(self.conv13(x))
        
        x = self.final_conv(x)

        return x





# Example usage:
model = UNet(input_channels=1, output_channels=2)
#input_tensor = torch.randn(1, 1, 128, 128)  # Adjust size as necessary
#y_pred = model(input_tensor)
#from torchviz import make_dot
# Visualize the model
#make_dot(y_pred, params=dict(model.named_parameters())).render("unet", format="png")
#print(model)

In [ ]:
# Split the data into training and testing sets
#data = data[..., np.newaxis]
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(y_train.shape)
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
# Set all values in x_train and x_test to be 0 for values lower than e-2
X_train[X_train < 1e-2] = 0
X_test[X_test < 1e-2] = 0
# Create the datasets and dataloaders
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
#print(train_dataset[0][0].shape)

# Get a batch of data
inputs, targets = next(iter(train_loader))
# Print the shape of the inputs
print(inputs.shape)

In [ ]:
#Different torch loss functions
class AbsoluteLogDifferenceLoss(nn.Module):
    def __init__(self):
        super(AbsoluteLogDifferenceLoss, self).__init__()

    def forward(self, y_pred, y_true):
        # Ensure that we avoid log of zero by adding a small epsilon
        epsilon = 1e-6
        # if y_pred is negative set it to 0
        y_pred = torch.clamp(y_pred, min=0)
        # if y_pred is infite set it to 0
        log_diff0 = torch.abs(torch.log(y_pred[:,0,:] + epsilon) - torch.log(y_true[:,0,:] + epsilon))
        log_diff1 = torch.abs(torch.log(y_pred[:,1,:] + epsilon) - torch.log(y_true[:,1,:] + epsilon))
        
        log_diff = log_diff0 + log_diff1
        #print(log_diff)
        regulator = 1
        #penalty = torch.abs(torch.log(y_pred[:,0,:])) + torch.abs(torch.log(y_pred[:,1,:])) - (torch.abs(torch.log(y_true[:,0,:])) + torch.abs(torch.log(y_true[:,1,:])))
        penalty = torch.abs(y_pred[:,2,:])
        #print(penalty)
        #+ LA.matrix_norm(regulator*penalty, ord = 'fro')
        # Use the L2 norm to calulate the loss
        loss = torch.mean(log_diff) + torch.mean(regulator*penalty)
        #print(np.shape(loss))
        return loss

class KLdivLoss(nn.Module):
    def __init__(self):
        super(KLdivLoss, self).__init__()

    def forward(self, y_pred, y_true):

        kl_div0 = F.kl_div(y_pred[:,0,:], y_true[:,0,:], reduction='batchmean')
        kl_div1 = F.kl_div(y_pred[:,1,:], y_true[:,1,:], reduction='batchmean')
        kl_div = torch.sum(kl_div0 + kl_div1)
        return kl_div

class L1Loss(nn.Module):
    def __init__(self):
        super(L1Loss, self).__init__()

    def forward(self, y_pred, y_true,X):
        l1_loss0 = F.l1_loss(y_pred[:,0,:], y_true[:,0,:])
        l1_loss1 = F.l1_loss(y_pred[:,1,:], y_true[:,1,:])
        #y_pred = torch.clamp(y_pred, min=0)
        sq_X = torch.squeeze(X)
        summed_spectogram = torch.sum(y_pred, dim=1)
        diff = torch.abs(torch_normalize(torch.squeeze(summed_spectogram)) - torch_normalize(sq_X))
        #penalty = torch.abs((y_pred[:,0,:]) + torch.abs(y_pred[:,1,:])) - (torch.abs(y_true[:,0,:]) + torch.abs(y_true[:,1,:]))
        l1_loss = torch.mean(l1_loss0 + l1_loss1) + torch.mean(diff)
        return l1_loss


In [ ]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_channels = X_train.shape[1]  # Get the number of input channels from the data
print(input_channels)
model = UNet(input_channels=input_channels,output_channels=2).to(device, dtype=torch.float32)
criterion = L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 10
best_loss = np.inf
patience = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X, y in train_loader:
        X, y = X.to(device, dtype=torch.float32), y.to(device, dtype=torch.float32)
        
        #print(f"Shape of x : {X.shape},Shape of y {y.shape}")
        optimizer.zero_grad() 
        y_pred = model(X)
        loss = criterion(y_pred, y, X)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0) # Accumulate loss
        #print(f"Loss: {loss.item()}")
    val_loss = 0.0
    model.eval()
    #Implement early stopping
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device, dtype=torch.float32), y.to(device, dtype=torch.float32)
            y_pred = model(X)
            loss = criterion(y_pred, y, X)
            
            val_loss += 4*loss.item() * X.size(0)
    if val_loss < best_loss:
        best_loss = val_loss
        best_model = model.state_dict()
        patience = 0
    else:
        patience += 1
    if patience > 5:
        break
    

    epoch_loss = running_loss
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}')
   
torch.save(model.state_dict(), 'your_model_weights.pth')

In [ ]:
#Generate a random waveform
instrument_list = [bass_electronic, flute_acoustic]
#Pick a random sample of each instrument
filepaths, labels = pick_samples_and_classify(instrument_list)

#Extract .wav data into to a list using librosa
waveforms = add_waveform_to_list(filepaths)
print(waveforms[0])
#Combine the waveforms
combined_waveform = combine_waveforms(waveforms)

# Convert the combined waveform to a spectrogram
spectrogram = waveform_to_spectogram(combined_waveform)
# Set all values lower than e-2 to 0
spectogram_0 = np.where(spectrogram < 1e-2, 0, spectrogram)
# Normalize the spectrogram
normalized_spectrogram = normalize_spectrogram(spectrogram)
normalized_spectrogram_0 = normalize_spectrogram(spectogram_0)
# Convert the normalized spectrogram back to audio and save it
spectogram_to_audio(normalized_spectrogram, 16000, "combo1.wav")
spectogram_to_audio(normalized_spectrogram_0, 16000, "combo2.wav")

In [ ]:
# Plot the spectrograms for the individual waveforms
sr = 16000  # Assuming a sample rate of 16kHz

plot_spectrogram(waveforms[0], sr, 'Spectrogram of First Waveform')
plot_spectrogram(waveforms[1], sr, 'Spectrogram of Second Waveform')

# If you want to plot the spectrogram of the combined waveform as well
plot_spectrogram(combined_waveform, sr, 'Spectrogram of Combined Waveform')

In [ ]:
import torch
import numpy as np
import scipy.signal as signal
import wave
import soundfile as sf
import librosa
# Initialize and load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(input_channels=1,output_channels=2).to(device, dtype=torch.float32)
model.load_state_dict(torch.load('your_model_weights.pth', map_location=device))
model.eval()


with wave.open('combo1.wav', 'rb') as wf:
    input_audio = wf.readframes(-1)
    input_sr = wf.getframerate()

#Load the combined audio waveform
combined_waveform, input_sr = librosa.load('combo1.wav', sr=None)

# Compute spectrogram using scipy signal
#f, t, Zxx = signal.spectrogram(combined_waveform, fs=input_sr, nperseg=256)
spectrogram = waveform_to_spectogram(combined_waveform)

# Convert spectrogram to PyTorch tensor
input_tensor = torch.tensor(spectrogram, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device, dtype=torch.float32)  # Add batch and channel dimensions
#print(input_tensor[0])
# Pass input through the model to get predictions
with torch.no_grad():
    output_tensor = model(input_tensor)

# Convert output tensor to numpy array and save as audio files
output_tensor = output_tensor.squeeze().cpu().numpy()
#print(output_tensor[0])
# Plot the separated spectrograms
plot_separated_spectrograms(output_tensor, input_sr)
output_folder = "output_audio"
#print(output_tensor)
# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert each source back from spectrogram to audio and save
for i, source in enumerate(output_tensor):
    # Convert the spectrogram back to audio

    # Save as .wav file
    output_filepath = os.path.join(output_folder, f'source_{i}.wav')
    spectogram_to_audio(source, input_sr, output_filepath)
    print(f"Saved {output_filepath}")

In [ ]:
#Plot the spectgram of the sum separated audio files subtracted from the original audio file

# Load the original audio file
original_waveform, sr = librosa.load('combo1.wav', sr=None)

# Load the separated audio files
separated_waveforms = []
for i in range(2):
    separated_waveform, sr = librosa.load(f'output_audio/source_{i}.wav', sr=None)
    separated_waveforms.append(separated_waveform)

# Sum the separated waveforms
sum_separated_waveform = np.sum(separated_waveforms, axis=0)

# Subtract the sum of the separated waveforms from the original waveform
diff_waveform = original_waveform - sum_separated_waveform

# Compute the spectrogram of the difference waveform
diff_spectrogram = waveform_to_spectogram(diff_waveform)

# Plot the spectrogram of the difference waveform

plot_spectrogram(diff_waveform, sr, 'Spectrogram of Difference Waveform')